In [1]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

In [107]:
x,y = load_diabetes(return_X_y=True,as_frame=True)

In [108]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2)

In [109]:
lr = LinearRegression()

In [110]:
lr.fit(x_train,y_train)

LinearRegression()

In [111]:
y_pred = lr.predict(x_test)

In [112]:
# my lr using mini batch gradient descent (mbgd)

In [151]:
class myMBGD:

    def __init__(self,batch_size = 25,rate=0.1,epochs=100 , lr_schedule = 0 ):
        self.intercept_ = None
        self.coef_ = None
        self.rate_ = rate
        self.init_rate_ = rate
        self.epochs_ = epochs
        self.batch_size_ = batch_size
        if not (0<= lr_schedule <1):
            raise ValueError("Learning schedule should be between 0 and 1")
        self.lr_schedule_ = lr_schedule

    def fit(self,x_train,y_train):
        n , num_of_coef = x_train.shape
        self.coef_ = np.ones(num_of_coef)
        self.intercept_ = 0
        
        for i in range(self.epochs_):
            
            self.rate_ = self.init_rate_ * (1 - self.lr_schedule_ * (i / self.epochs_))
            
            shuffled_indices = np.random.permutation(len(x_train))
            x_train = x_train.iloc[shuffled_indices].reset_index(drop=True)
            y_train = y_train.iloc[shuffled_indices].reset_index(drop=True)
                
            for j in range(0, n ,self.batch_size_):
                
                x_train_temp = x_train.iloc[j : j + self.batch_size_]
                y_train_temp = y_train.iloc[j : j + self.batch_size_]
                divisor = x_train_temp.shape[0]
                
                y_hat = np.dot(x_train_temp,self.coef_.T) + self.intercept_
                
                intercept_der = -(2/divisor) * np.sum(y_train_temp-y_hat)
                self.intercept_ = self.intercept_ - (self.rate_*(intercept_der))
    
                coef_der = -(2/divisor) * np.dot((y_train_temp-y_hat).T,x_train_temp)
                self.coef_ = self.coef_ - (self.rate_ * coef_der)


    def predict(self,x_test):
        
        return np.dot(x_test,self.coef_.T) + self.intercept_
                

In [231]:
mymbgd = myMBGD(epochs = 150 , rate = 0.5 , batch_size = 30)

In [232]:
mymbgd.fit(x_train,y_train)

In [233]:
print(mymbgd.coef_,mymbgd.intercept_)

[ -32.18817758 -212.74707832  513.31549427  267.66948796  -64.74460958
 -119.061138   -201.4024135   130.91234391  439.15394125  148.37374338] 122.34370209481644


In [234]:
print(lr.coef_ , lr.intercept_)

[ -40.84716608 -220.27793989  518.65916767  265.72780681 -899.42124132
  538.75301609  163.021775    222.18950804  768.53189151  136.84001643] 153.58425908610226


In [235]:
y_pred_mymbgd = mymbgd.predict(x_test)

In [236]:
r2_score(y_test,y_pred_mymbgd)

0.4276354185459599

In [237]:
r2_score(y_test,y_pred)

0.518135411935364

In [121]:
x_train.shape

(353, 10)